In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install llama-index langchain_community "httpx==0.27.2"

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [ ]:
from llama_index.core.tools import FunctionTool
import nest_asyncio

nest_asyncio.apply()

def multiply(a: int, b: int) -> int:
    return a * b

# multyply 함수를 곱하기 도구로 등록
multiply_tool = FunctionTool.from_defaults(fn = multiply)

def divide(a: int, b: int) -> int:
    return a / b

# divide 함수를 나누기 도구로 등록
divide_tool = FunctionTool.from_defaults(fn = divide)
tools = [multiply_tool, divide_tool] # 등록된 도구들을 리스트로 관리

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
agent_engine = OpenAIAgentWorker.from_tools(
    tools,
    llm = llm,
    verbose = True
) # 도구와 LLM을 겳합하여 에이전트를 생성
agent = AgentRunner(agent_engine)
# "(121 * 12) / 4 결과는?"을 작업으로 생성
task = agent.create_task("(121*12)/4 결과는?")

# 모든 단계 실행
# 작업 ID를 기반으로 아직 실행되지 않은 단계(step)들을 반환
while len(agent.get_upcoming_steps(task.task_id)) > 0:
    agent.run_step(task.task_id) # 한 번에 하나의 단계를 실행

# 최종 응답 확인
# 모든 단계가 완료되었는지 확인한 후, 작업의 최종 결과를 생성
response = agent.finalize_response(task.task_id)
print(f"Final Response: {response}")

In [ ]:
!pip install llama-index python-dotenv "httpx==0.27.2"

In [ ]:
import os
import nest_asyncio

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
nest_asyncio.apply() # 중첩 실행 허용

In [ ]:
from llama_index.llms.openai import OpenAI

# LLM 설정
llm = OpenAI(model="gpt-4o") # 원하는 LLM 모델 설정

In [ ]:
# '/content/drive/MyDrive/store/' 은 PDF 파일이 위치한 경로입니다.
pdf_file = SimpleDirectoryReader(
    input_dir = "/content/drive/MyDrive/store"
).load_data()

In [ ]:
# 문서를 벡터로 변환하여 저장
vector_query_engine = VectorStoreIndex.from_documents(
    pdf_file,
    use_async=True,
).as_query_engine( ) # VectorStoreIndex를 기반으로 질의 응답 기능을 활성화

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine = vector_query_engine,
        metadata = ToolMetadata(
            name = "Q&A bot",
            description = "당신은 사용자 질문에 답변하는 봇입니다.",
        ),
    ),
]   # 사용자 질문에 대한 답변 생성

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools = query_engine_tools,
    llm = llm,  # 추가된 LLM 사용
    use_async = True,
)   # 복잡한 질문을 하위 질문(Sub-Question)으로 분리하여 처리하고 결과를 조합

In [ ]:
response = query_engine.query(
    "임산부를 위한 운동에는 어떤 운동이 있는지?"
)
print(response)